In [2]:
#data 가져오기, numpy, tensorflow없이 pandas써도 상관없나?

import pandas as pd
data = pd.read_csv("../data/creditcard.csv") #dataframe의 index가 필요하다
print(data.shape)
data.head()
data.dtypes
#type(data) = pandas.core.frame.DataFrame

(284807, 31)


Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object


## train,test data 분할하기


In [17]:
#import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
y_data= data[["Class"]]
#y_data.head()
x_data=data[['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']]
#data.columns

print(x_data.shape, y_data.shape)
'''
x_data=data.loc[:,"Time"]
print(x_data.shape)
print(y_data.shape)

x_data= data[:,0:-1]
y_data= data[:,[-1]]

import random
ranint = random.sample(range(0,284806), 2000)
test= data.iloc[ranint,:]
train = data.iloc[ranint,:] #제외한 나머지 명령어는 뭐지?
print(test.shape, train.shape)

#train = train[ranint]
#print(train.shape)
'''
x_train,x_test, y_train,y_test = train_test_split(x_data,y_data, test_size=0.2,random_state=42)
print(x_train.shape,x_test.shape, y_train.shape,y_test.shape)

(284807, 30) (284807, 1)
(227845, 30) (56962, 30) (227845, 1) (56962, 1)


## decision tree model


In [19]:
#scikit-learn == sklearn
#sklearn.tree.DecisionTreeClassifier 힘들다
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=5)
model
model.fit(x_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [95]:
predictions = model.predict(x_test)
print(predictions.shape)

'''
from pandas import DataFrame
predict = DataFrame(predictions)
predict.columns=['predict']
#predictions[0:10]
print(predict.shape)

import numpy as np
np_ytest = y_test.values
print(np_ytest.shape)

print(predict.loc[0])
print(y_test.loc[0])
predict.head()
#y_test.head()
'''
type(y_test)
ytest=y_test.values
type(ytest)
#type(predictions)

(56962,)


numpy.ndarray

In [ ]:

accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions,Y ), dtype=tf.float32))

np_ytest = y_test.values
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    # Initialize TensorFlow variables

    # Accuracy report
    a = sess.run(accuracy, feed_dict={ Y:ytest})
    print("\nAccuracy: ", a)

In [100]:
acc = 0
for i in range(56961):
    if predictions[i]==ytest[i]:
        acc = acc+1
        
print(acc/56962)

0.9994733330992591


## Gradient Descent Optimizer
_data가 크니 x_train만 하자.

In [13]:
# placeholders for a tensor that will be always fed.

X = tf.placeholder(tf.float32, shape=[None, 30])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([30, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)  # y hat value


# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

#train = tf.train.GradientDescentOptimizer(learning_rate=0.02).minimize(cost)

#adam optimizer
train = tf.train.AdamOptimizer(1e-2).minimize(cost)

# Accuracy computation

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)# True if hypothesis>0.5 else False
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

print('well')


well


In [16]:
# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_train, Y: y_train})
        if step % 500 == 0:
            print(step, cost_val)
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_train, Y: y_train})
    print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)
    print(h.shape,"  ", c.shape)

    '''
    session도 같은 그룹안에서 사용해야 한다. 
    cell추가해서 따로 하면 안됨. 지역변수라서.
    '''
 

0 nan
200 nan
400 nan
600 nan
800 nan
1000 nan
1200 nan
1400 nan
1600 nan
1800 nan
2000 nan


KeyboardInterrupt: 